Install necessary Fake useragent

In [1]:
pip install fake_useragent pandas


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Import necessary libraries

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

Genereating Headers

In [3]:
url = 'https://www.worldfootball.net/top_scorer/eng-premier-league//'
ua = UserAgent()
userAgent = ua.random
headers = {'User-Agent' : userAgent}
jumia_page= requests.get(url, headers= headers)
soup = BeautifulSoup(jumia_page.content, 'html.parser')
print(jumia_page)

<Response [200]>


Scraping Top Scorers Table

In [4]:
# scraping the table
Football_table = soup.find('table', class_="standard_tabelle")

# scraping the table headers
table_head = soup.find_all('td', class_="ueberschrift")
cleaned_table_head = [cell.text.strip() for cell in table_head]

# Scraping table rows
rows = Football_table.find_all("tr")

data = []
cleaned_table_rows=[]

# Loop through the rows
for i in rows:
    table_rows = i.find_all("td", class_=["hell", "dunkel"])  # Find table rows with specific classes
    cleaned_table_rows = [ell.text.strip() for ell in table_rows]

    # Extract country name from the image alt attribute
    for t in table_rows:
        country_img = t.find("img")
        if country_img:
            Country = country_img.get("alt", "").strip()
            cleaned_table_rows.append(Country)
    
    data.append(cleaned_table_rows)

# Insert the table headers into the table i.e. data
data.insert(0, cleaned_table_head)

# Updating the table headers to account for empty columns
data[0] = ['Season', 'None', 'Top scorer', 'None2', 'Team', 'goals', 'Country', 'None3'] 

print(data)
print(cleaned_table_rows)



[['Season', 'None', 'Top scorer', 'None2', 'Team', 'goals', 'Country', 'None3'], [], ['2023/2024', '', 'Erling Haaland', '', 'Manchester City', '27', 'Norway', 'Manchester City'], ['2022/2023', '', 'Erling Haaland', '', 'Manchester City', '36', 'Norway', 'Manchester City'], ['2021/2022', '', 'Mohamed Salah', '', 'Liverpool FC', '23', 'Egypt', 'Liverpool FC'], ['', '', 'Heung-min Son', '', 'Tottenham Hotspur', '23', 'South Korea', 'Tottenham Hotspur'], ['2020/2021', '', 'Harry Kane', '', 'Tottenham Hotspur', '23', 'England', 'Tottenham Hotspur'], ['2019/2020', '', 'Jamie Vardy', '', 'Leicester City', '23', 'England', 'Leicester City'], ['2018/2019', '', 'Sadio Mané', '', 'Liverpool FC', '22', 'Senegal', 'Liverpool FC'], ['', '', 'Pierre-Emerick Aubameyang', '', 'Arsenal FC', '22', 'Gabon', 'Arsenal FC'], ['', '', 'Mohamed Salah', '', 'Liverpool FC', '22', 'Egypt', 'Liverpool FC'], ['2017/2018', '', 'Mohamed Salah', '', 'Liverpool FC', '32', 'Egypt', 'Liverpool FC'], ['2016/2017', '', 'H

In [5]:
# save data into a dataframe
Top_scorers_table = pd.DataFrame(data)

# make first rows the header
if not Top_scorers_table.empty:
    Top_scorers_table.columns = Top_scorers_table.iloc[0]  # Use the first row as header
    Top_scorers_table = Top_scorers_table[1:]

# drop empty columns
Top_scorers_table.drop(['None', 'None2', 'None3'], axis=1, inplace=True)

# drop first empty row that is not the header
Top_scorers_table.drop(1, inplace=True)
first_5_rows = Top_scorers_table[:5]

# print Top scorers table
print(Top_scorers_table)


0       Season       Top scorer               Team goals      Country
2    2023/2024   Erling Haaland    Manchester City    27       Norway
3    2022/2023   Erling Haaland    Manchester City    36       Norway
4    2021/2022    Mohamed Salah       Liverpool FC    23        Egypt
5                 Heung-min Son  Tottenham Hotspur    23  South Korea
6    2020/2021       Harry Kane  Tottenham Hotspur    23      England
..         ...              ...                ...   ...          ...
143  1892/1893  Johnny Campbell     Sunderland AFC    31     Scotland
144  1891/1892  Johnny Campbell     Sunderland AFC    32     Scotland
145  1890/1891  Jack Southworth   Blackburn Rovers    26      England
146  1889/1890       Jimmy Ross  Preston North End    21     Scotland
147  1888/1889     John Goodall  Preston North End    21      England

[146 rows x 5 columns]
